## System Setup

In [1]:
# Find CPU info
!grep -m1 'model name' /proc/cpuinfo | awk -F": " '{print $2}'
!grep 'cpu cores' /proc/cpuinfo | awk -F": " '{a[cores]+=$2}END{print "CPU cores: " a[cores]}'

Intel(R) Xeon(R) CPU @ 2.20GHz
CPU cores: 8


In [2]:
# Find Ram Info
!grep MemTotal /proc/meminfo | awk '{printf "%.1fGB RAM", $2 / 1024 / 1024}'

25.5GB RAM

In [3]:
!python3 --version

Python 3.10.6


In [4]:
!nvidia-smi

Fri Jul 21 05:17:29 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   39C    P0    26W / 300W |      0MiB / 16384MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [5]:
!pip install pyBKTR

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 220.3/220.3 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.9/88.9 kB 4.0 MB/s eta 0:00:00


## BIXI Data Analysis (Python)

In [6]:
from pyBKTR.bktr import BKTRRegressor
from pyBKTR.examples.bixi import BixiData
from pyBKTR.kernels import (
    KernelMatern, KernelParameter,
    KernelPeriodic, KernelSE)
from pyBKTR.plots import (

    plot_hyperparams_traceplot, plot_hyperparams_dists
)
from pyBKTR.tensor_ops import TSR

df_res_arr = []
TSR.set_params('float32', 'cuda', 1)
bixi_data = BixiData()

fm = 'nb_departure ~ humidity + walkscore + total_precip_mm'
p_len = KernelParameter(value = 7, is_fixed = True)
k_loc_per = KernelSE() * KernelPeriodic(period_length = p_len)

bktr_regressor = BKTRRegressor(
    formula = fm,
    data_df = bixi_data.data_df,
    spatial_positions_df = bixi_data.spatial_positions_df,
    temporal_positions_df = bixi_data.temporal_positions_df,
    rank_decomp = 8,
    spatial_kernel = KernelMatern(smoothness_factor = 5),
    temporal_kernel = k_loc_per,
    burn_in_iter = 1000,
    sampling_iter = 500
)
bktr_regressor.mcmc_sampling()

* Iter 1     | Elapsed     1.29s | MAE  0.0997 | RMSE  0.1367 *
* Iter 2     | Elapsed     0.10s | MAE  0.0637 | RMSE  0.0865 *
* Iter 3     | Elapsed     0.11s | MAE  0.0581 | RMSE  0.0791 *
* Iter 4     | Elapsed     0.29s | MAE  0.0570 | RMSE  0.0774 *
* Iter 5     | Elapsed     0.29s | MAE  0.0565 | RMSE  0.0768 *
* Iter 6     | Elapsed     0.19s | MAE  0.0563 | RMSE  0.0764 *
* Iter 7     | Elapsed     0.34s | MAE  0.0563 | RMSE  0.0763 *
* Iter 8     | Elapsed     0.23s | MAE  0.0560 | RMSE  0.0760 *
* Iter 9     | Elapsed     0.35s | MAE  0.0560 | RMSE  0.0760 *
* Iter 10    | Elapsed     0.27s | MAE  0.0558 | RMSE  0.0757 *
* Iter 11    | Elapsed     0.27s | MAE  0.0558 | RMSE  0.0757 *
* Iter 12    | Elapsed     0.16s | MAE  0.0557 | RMSE  0.0755 *
* Iter 13    | Elapsed     0.26s | MAE  0.0557 | RMSE  0.0755 *
* Iter 14    | Elapsed     0.21s | MAE  0.0556 | RMSE  0.0754 *
* Iter 15    | Elapsed     0.25s | MAE  0.0556 | RMSE  0.0754 *
* Iter 16    | Elapsed     0.24s | MAE  

In [7]:
print(bktr_regressor.summary)


                        BKTR Regressor Summary                        
Formula: nb_departure ~ 1 + humidity + walkscore + total_precip_mm

Burn-in iterations: 1000
Sampling iterations: 500
Rank decomposition: 8
Nb Spatial Locations: 587
Nb Temporal Points: 196
Nb Covariates: 4
In Sample Errors:
  RMSE: 0.072
  MAE: 0.053
Computation time: 523.54s.
-- Spatial Kernel --
Matern 5/2 Kernel
Parameter(s):
                       Mean   Median       SD  Low2.5p  Up97.5p
lengthscale          22.364   22.513    1.931   18.925   26.711

-- Temporal Kernel --
Composed Kernel (Mul)
  SE Kernel
  Parameter(s):
                         Mean   Median       SD  Low2.5p  Up97.5p
  lengthscale           6.641    6.636    0.107    6.438    6.881
  *
  Periodic Kernel
  Parameter(s):
                         Mean   Median       SD  Low2.5p  Up97.5p
  lengthscale           1.012    1.012    0.019    0.973    1.048
  period length          Fixed Value: 7.000
Beta Estimates Summary (Aggregated Per Covariates

In [8]:
from pyBKTR.plots import plot_hyperparams_traceplot
plot_hyperparams_traceplot(bktr_regressor, [
  'Spatial - Matern 5/2 Kernel - lengthscale',
  'Temporal - SE Kernel - lengthscale'])

In [9]:
from pyBKTR.plots import plot_hyperparams_dists
plot_hyperparams_dists(bktr_regressor, [
  'Spatial - Matern 5/2 Kernel - lengthscale',
  'Temporal - SE Kernel - lengthscale'])

In [10]:
from pyBKTR.plots import plot_temporal_betas
plot_temporal_betas(
  bktr_regressor,
  plot_feature_labels = ['humidity', 'walkscore', 'total_precip_mm'],
  spatial_point_label = '7114 - Smith / Peel', show_figure=False)

In [11]:
from pyBKTR.plots import plot_spatial_betas
plot_spatial_betas(
  bktr_regressor,
  plot_feature_labels = ['humidity', 'walkscore', 'total_precip_mm'],
  temporal_point_label = '2019-07-19',
  nb_cols = 3, show_figure=True, fig_width=850, fig_height=400)

In [12]:
b_df = bktr_regressor.get_beta_summary_df(
    spatial_labels=['7114 - Smith / Peel'],
    temporal_labels=['2019-07-18', '2019-07-19'],
    feature_labels=None
)
print('Columns: ' + ' '.join(b_df.columns.to_list()))
print()
b_df = b_df[['Mean', 'SD', 'Low2.5p', 'Up97.5p']]
b_df = b_df.droplevel(0)
print(b_df.to_string(float_format='{:.2f}'.format))

Columns: Mean SD Min Q1 Median Q3 Max Low2.5p Up97.5p

                            Mean   SD  Low2.5p  Up97.5p
2019-07-18 Intercept        0.89 0.06     0.78     0.99
           humidity        -0.37 0.03    -0.44    -0.29
           walkscore        0.30 0.07     0.18     0.44
           total_precip_mm -0.52 0.04    -0.60    -0.45
2019-07-19 Intercept        0.85 0.06     0.72     0.97
           humidity        -0.35 0.04    -0.42    -0.26
           walkscore        0.37 0.08     0.23     0.54
           total_precip_mm -0.51 0.03    -0.57    -0.45
